## Parameters

In the next two cells, we import the required Python libraries, provide the access to instances and set up the sets of sorting rules.

In [1]:
import pandas as pd
import os

In [2]:
# Path to problem instances and list their names
instance_path_Ap = 'MLCLSP Instances/Ap/'
instance_path_Bp = 'MLCLSP Instances/Bp/'
file_names_Ap = [file for file in os.listdir(instance_path_Ap) if file.endswith('.dat')]
file_names_Bp = [file for file in os.listdir(instance_path_Bp) if file.endswith('.dat')]

# Criteria for sorting rules
cost_crit = ['CUI', 'EC','ES','ESC', 'H', 'HS', 'HSC', 'S','SH','TBO']
period_crit = ['CUP', 'DECP','INCP']
capacity_crit = ['D','DH', 'DHS', 'DS', 'DSH']
key_crit = ['IP', 'PI']

# All sorting rules (75)
sorting_rules = []
for pc in period_crit:
    for ic in cost_crit:
        for k in key_crit:
            rule = pc+'-'+ic+'-'+k
            sorting_rules.append(rule)
for pc in period_crit:
    for dc in capacity_crit:
        rule = dc+'-'+k
        sorting_rules.append(rule)

# Reduced set of soring rules (29)
reduced_set = ['CUP-CUI-IP', 'CUP-DH', 'CUP-DS', 'CUP-ESC-PI', 'CUP-ES-IP', 'CUP-H-IP', 'CUP-SH-PI', 'CUP-TBO-IP',
               'DECP-CUI-IP', 'INCP-CUI-IP', 'INCP-CUI-PI', 'INCP-D', 'INCP-DH', 'INCP-DHS', 'INCP-DS', 'INCP-DSH',
               'INCP-EC-IP', 'INCP-EC-PI', 'INCP-ESC-IP', 'INCP-ESC-PI', 'INCP-ES-PI', 'INCP-H-IP', 'INCP-H-PI',
               'INCP-HSC-IP', 'INCP-HS-PI', 'INCP-SH-IP', 'INCP-SH-PI', 'INCP-S-IP', 'INCP-TBO-IP']

In [3]:
print('Parameters for 2-SCH\n')
print('Path to instance files:\n', instance_path_Ap, '\n', instance_path_Bp)
print('\nComplete set of sorting rules:\n', sorting_rules)
print('\nReduced set of sorting rules:\n', reduced_set)

Parameters for 2-SCH

Path to instance files:
 MLCLSP Instances/Ap/ 
 MLCLSP Instances/Bp/

Complete set of sorting rules:
 ['CUP-CUI-IP', 'CUP-CUI-PI', 'CUP-EC-IP', 'CUP-EC-PI', 'CUP-ES-IP', 'CUP-ES-PI', 'CUP-ESC-IP', 'CUP-ESC-PI', 'CUP-H-IP', 'CUP-H-PI', 'CUP-HS-IP', 'CUP-HS-PI', 'CUP-HSC-IP', 'CUP-HSC-PI', 'CUP-S-IP', 'CUP-S-PI', 'CUP-SH-IP', 'CUP-SH-PI', 'CUP-TBO-IP', 'CUP-TBO-PI', 'DECP-CUI-IP', 'DECP-CUI-PI', 'DECP-EC-IP', 'DECP-EC-PI', 'DECP-ES-IP', 'DECP-ES-PI', 'DECP-ESC-IP', 'DECP-ESC-PI', 'DECP-H-IP', 'DECP-H-PI', 'DECP-HS-IP', 'DECP-HS-PI', 'DECP-HSC-IP', 'DECP-HSC-PI', 'DECP-S-IP', 'DECP-S-PI', 'DECP-SH-IP', 'DECP-SH-PI', 'DECP-TBO-IP', 'DECP-TBO-PI', 'INCP-CUI-IP', 'INCP-CUI-PI', 'INCP-EC-IP', 'INCP-EC-PI', 'INCP-ES-IP', 'INCP-ES-PI', 'INCP-ESC-IP', 'INCP-ESC-PI', 'INCP-H-IP', 'INCP-H-PI', 'INCP-HS-IP', 'INCP-HS-PI', 'INCP-HSC-IP', 'INCP-HSC-PI', 'INCP-S-IP', 'INCP-S-PI', 'INCP-SH-IP', 'INCP-SH-PI', 'INCP-TBO-IP', 'INCP-TBO-PI', 'D-PI', 'DH-PI', 'DHS-PI', 'DS-PI', 'DSH-PI

### 2-SCH Main code
In the next cells, we load the Cython Jupyter extension and define classes and methods for 2-SCH in Cython. 

In [4]:
%load_ext cython

In [5]:
%%cython
# cython: boundscheck=False, wraparound=False, initializedcheck=False, nonecheck=False, cdivision=True

DEF MyAbsTol = 1e-12
DEF MyRelTol = 1e-5

import numpy as np
cimport numpy as np
from libc.math cimport fmin, fmax, sqrt, round, ceil
ctypedef np.uint8_t uint8
import random

cdef class DemandList:
    """Class for a list of demand elements.
    It provides a fast way of handling a list of demand elements

    Data:
        length (int): number of elements in the demand list (= length of arrays)
        item_index (c_item_index) (int-ndarray): array of item indices ordered according to the demand list (C memoryview)
        period_index (c_period_index) (int-ndarray): array of period indices ordered according to the demand list (C memoryview)
        amounts (c_amounts) (double-ndarray): array of demand amounts ordered according to the demand list (C memoryview)
        
        (item_index[n], period_index[n], amounts[n]) = n-th demand element

    Methods:
        __init__(length=0): initialize the class and the arrays of the given length.
        load_from_demandMatrix (demand, nr_items, nr_periods): initialize demand list from the demand matrix
        load_from_p_DemandList (demand, p_DemandList): initialize demand list from the python list
        copy (pos1, pos2): return a copy of the demand list between positions pos1 and po2
        swap (pos1, pos2): swap demand elements between positions pos1 and pos2
        get_p_DemandList (): return a python demand list """

    # definition of class variables 
    cdef public np.ndarray item_index, period_index, amounts, cost_upto
    cdef public int length
    cdef int[:] c_item_index, c_period_index
    cdef double[:] c_amounts, c_cost_upto
    
    # constructor
    # set up data structures for a specific length
    def __init__(self, int length = 0):
        self.length = length
        self.item_index = np.zeros(self.length, dtype=int)
        self.period_index = np.zeros(self.length, dtype=int)
        self.amounts = np.zeros(self.length, dtype=float)
        self.c_item_index = self.item_index
        self.c_period_index = self.period_index
        self.c_amounts = self.amounts

    # load data from demand matrix
    cdef void load_from_demandMatrix (self, double[:,:] demand, int nr_items, int nr_periods):
        cdef int i, t
        cdef int pos = 0
        if (nr_items*nr_periods != self.length):             # count nonzero demand entries in the matrix 
            self.length = 0
            for i in range(nr_items):
                for t in range (nr_periods):
                    if demand[i,t] > MyAbsTol:
                        self.length += 1
            self.__init__(self.length)
        for i in range(nr_items):                           # build demand list
            for t in range (nr_periods):
                if demand[i,t]>0:
                    self.c_item_index[pos] = i
                    self.c_period_index[pos] = t
                    self.c_amounts[pos] = demand[i,t]
                    pos += 1
                    
    # load data from a python demand list [(i,t)] and a demand matrix
    cpdef void load_from_p_DemandList (self, double[:,:] demand, list p_DemandList):
        cdef int pos
        if (len(p_DemandList) != self.length):
            self.__init__(len(p_DemandList))
        for pos in range(len(p_DemandList)):
            self.c_item_index[pos] = <int> p_DemandList[pos][0]
            self.c_period_index[pos] = <int> p_DemandList[pos][1]
            self.c_amounts[pos] = <double> demand[p_DemandList[pos][0],p_DemandList[pos][1]]
    
    # create a copy of the demand list
    cpdef DemandList copy (self, int pos1=0, int pos2=0):
        dl = DemandList(self.length)
        if pos2 == 0:
            pos2 = self.length
        dl.c_item_index[pos1:pos2] = self.c_item_index[pos1:pos2]
        dl.c_period_index[pos1:pos2] = self.c_period_index[pos1:pos2]
        dl.c_amounts[pos1:pos2] = self.c_amounts[pos1:pos2]
        return dl
        
    # swap elements on positions pos1 and pos2 in the demand list
    cdef void swap(self,int pos1, int pos2):
        self.c_item_index[pos1], self.c_item_index[pos2] = self.c_item_index[pos2], self.c_item_index[pos1]
        self.c_period_index[pos1], self.c_period_index[pos2] = self.c_period_index[pos2], self.c_period_index[pos1]
        self.c_amounts[pos1], self.c_amounts[pos2] = self.c_amounts[pos2], self.c_amounts[pos1]
    
    # get a python demand list
    def get_p_DemandList(self):
        cdef int pos
        dl = []
        for pos in range(self.length):
            dl.append((self.c_item_index[pos], self.c_period_index[pos]))
        return dl
    
    
cdef class MLCLSP_data:
    """Class for multi-level capacitated lot sizing problems instances.
    Can load data from text files (Stadtler instances).
    
    Data:
        instanceName (string): name of the loaded instance
        averageTBO_from_file (string): TBO level read from data file
        
        items (int): number of items
        periods (int): number of periods
        machines (int): number of machines
        nonzerodemand (int): number of nonzero (> 1e-12) entries in the demand matrix (only external demand on the current planning level)
        
        demand (numpy.array, dimension: items,periods): demand for each item on the current planning level and period
        capacity (numpy.array, dimension: machines, periods): capacities for each machine for each period
        initInventory (numpy.array, dimension: items): initial inventory for each item
        prodCoeffAll (numpy.array, dimension: machines, items): capacity consumption for each machine for each item
        prodCoeff (numpy.array, dimension: items): capacity consumption for each item
        holdingCost (numpy.array, dimension: items): holding cost for each item (per unit, per period)
        setupCost (numpy.array, dimension: items): setup cost for each item
        setupTime (numpy.array, dimension: machines, items): capacity consumption of setup operation for each machine for each item
        leadTime  (numpy.array, dimension: items): lead time for each item
        
        gozinto (numpy.array, dimension: items, items): item is an immediate predecessor of another item (=1)
        BOM (numpy.array, dimension: items, items): bill-of-material for all items
        extDemand (numpy.array, dimension: items, periods): external demand on the current planning level
        machineitem (numpy.array, dimension: machines, items): machine-item assignment
        itemmachine (numpy.array, dimension: items): item-machine assignment, machines are indexed as 0,1,2...

        
        TBO (numpy.array, dimension: items): TBO of each item
        averageDemand (numpy.array, dimension: items): average demand of each item
        allDemand (numpy.array, dimension: items, periods): demand for each item and period for all items
        
        averagePeriodDemand (double): avergae total demand per period 
        averageTBO (double): average TBO of all items
        
        overtimeCost (double): overtime cost per unit capacity (set to 10000, not loaded from instance file)
        
        c_... C memoryviews of numpy arrays

    Methods:
        __init__(path, filename, setuptimezero=True): initialize the class and load the data from the given instance file;
                                setup time is set to 0 (setuptimezero=True) or read from the file (setuptimezero=False). """
    
    cdef public str instanceName, averageTBO_from_file
    cdef public int items, periods, machines, nonzerodemand
    cdef public np.ndarray demand, capacity, initInventory, prodCoeffAll, prodCoeff, holdingCost, setupCost, setupTime, leadTime
    cdef public np.ndarray machineitem, gozinto, extDemand, itemmachine, BOM
    cdef public np.ndarray TBO, averageDemand, allDemand
    cdef public double averagePeriodDemand, capTightness, averageTBO, lumpiness, stdvDemand
    cdef public double overtimeCost
    cdef double[:,:] c_demand
    cdef double[:,:] c_capacity
    cdef double[:] c_initInventory
    cdef double[:,:] c_prodCoeffAll
    cdef double[:] c_prodCoeff
    cdef double[:] c_holdingCost
    cdef double[:] c_setupCost
    cdef double[:,:] c_setupTime
    cdef double[:] c_leadTime
    cdef double[:,:] c_gozinto
    cdef double[:,:] c_BOM
    cdef double[:,:] c_extDemand
    cdef uint8[:,:] c_machineitem
    cdef int [:] c_itemmachine
    cdef double[:] c_TBO
    cdef double[:] c_averageDemand
    cdef double[:,:] c_allDemand
        
    # initialize CLPS_data class - reads data from instnace file
    def __init__(self, path, filename, setuptimezero = True):
        extdemand = []; capacity = []; initinv = []; holdc = []; prodcoeff_all = []; prodcoeff = []; machineitem = []; setupc = []; setupt = []; leadt = [] # initialize lists
        with open(path + filename,"r") as iFile:               # read data from file
            self.instanceName = filename    # name of instance
            iFile.readline();iFile.readline();iFile.readline()          # skip 3 lines 
            vals = iFile.readline().strip().split()
            self.items = int(vals[1])
            self.periods = int(vals[0])
            self.machines = int(vals[2])
            iFile.readline()                        
            for i in range(0, self.items):                              # read setup cost, holding cost, lead time, init inv
                values = iFile.readline().strip().split()
                setupc.append(float(values[0]))
                holdc.append(float(values[1]))
                leadt.append(float(values[2]))
                initinv.append(float(values[3]))
            self.setupCost = np.array(setupc, dtype=float)
            self.c_setupCost = self.setupCost
            self.leadTime = np.array(leadt, dtype=float)
            self.c_leadTime = self.leadTime
            iFile.readline()
            gozinto = np.zeros((self.items, self.items))
            for i in range(0, self.items):                              # read gozinto factor
                values = iFile.readline().strip().split()
                for j in range(0, self.items):
                    gozinto[i][j] = float(values[j])
            iFile.readline()
            for i in range(0, self.items):                              # read external demand
                values = iFile.readline().strip().split()
                demand_i = []
                for val in values:
                    demand_i.append(float(val))
                extdemand.append(demand_i.copy())
            iFile.readline()
            for m in range(0, self.machines):                           # read capacity
                values = iFile.readline().strip().split()
                capacity_m = []
                for val in values:
                    capacity_m.append(float(val))
                capacity.append(capacity_m.copy())
            self.capacity = np.array(capacity)
            self.c_capacity = self.capacity
            iFile.readline()
            for m in range(0, self.machines):                           
                values = iFile.readline().strip().split()
                prodcoeff_m = []
                for val in values:
                    prodcoeff_m.append(float(val))
                prodcoeff_all.append(prodcoeff_m.copy())
            self.prodCoeffAll = np.array(prodcoeff_all, dtype=float)     # read all production coefficients                                             
            self.c_prodCoeffAll = self.prodCoeffAll
            machineitem = np.zeros((self.machines, self.items))
            for m in range(0, self.machines):                           
                for i in range(0, self.items):
                    if prodcoeff_all[m][i] > 0:
                        prodcoeff.append(prodcoeff_all[m][i])          
                        machineitem[m][i] = 1
            self.machineitem = np.array(machineitem, dtype=np.uint8)     # read which item is produced on which machine
            self.c_machineitem = self.machineitem
            self.itemmachine = np.zeros(self.items, dtype=int)
            for i in range(0, self.items):
                self.itemmachine[i] = np.argmax(self.machineitem[:,i])
            self.c_itemmachine = self.itemmachine
            self.prodCoeff = np.array(prodcoeff)                        # read only positive prodcoeff (1:1 item-machine relation)
            self.c_prodCoeff = self.prodCoeff
            self.holdingCost = np.array(holdc)                          
            self.holdingCost = self.holdingCost/self.prodCoeff          # convert holding cost to capacity units
            self.c_holdingCost = self.holdingCost
            for i in range(0, self.items):                              
                initinv[i] = initinv[i]*prodcoeff[i]                    # convert initial inventory to capacity units
            self.initInventory = np.array(initinv, dtype=float)
            self.c_initInventory = self.initInventory
            for i in range(0, self.items):                              
                for j in range(0, self.items):
                    gozinto[i][j] *= (prodcoeff[j]*prodcoeff[i])        # convert gozinto to capacity units
            self.gozinto = np.array(gozinto)    
            self.c_gozinto = self.gozinto
            self.BOM = np.array(gozinto)                                # compute BOM
            self.c_BOM = self.BOM
            for i in range(self.items-1,-1,-1):
                for pred in range(self.items-1,i,-1):
                    if self.BOM[pred,i] > 0:
                        for ppred in range(self.items-1,pred,-1):
                            self.BOM[ppred,i] += self.BOM[ppred,pred]*self.BOM[pred,i]   
            for i in range(0, self.items):                              
                for p in range(0, self.periods):
                    extdemand[i][p] = extdemand[i][p]*prodcoeff[i]      # convert external demand to capacity units
            self.demand = np.array(extdemand)
            self.c_demand = self.demand
            self.extDemand = np.array(extdemand)
            self.c_extDemand = self.extDemand
            iFile.readline()
            for m in range(0, self.machines):                           # read setup time
                values = iFile.readline().strip().split()
                setupt_m = []
                for val in values:
                    setupt_m.append(float(val))
                setupt.append(setupt_m.copy())
            self.setupTime = np.array(setupt)
            self.c_setupTime = self.setupTime
            self.overtimeCost = 10000                                   # set overtime cost to 10,000
            self.nonzerodemand = 0                                      # count non-zero demand elements
            for i in range(self.items):
                for t in range(0,self.periods):
                    if (self.c_extDemand[i][t] > MyAbsTol):
                        self.nonzerodemand += 1
            self.averageDemand = np.zeros(self.items, dtype=float)      # compute average demand
            self.c_averageDemand = self.averageDemand
            self.allDemand = np.zeros((self.items,self.periods), dtype=float)
            for i in range(self.items):
                for t in range(self.periods):
                    self.allDemand[i][t] = self.extDemand[i][t]
                    self.c_averageDemand[i] += self.c_extDemand[i][t]
                    for j in range(self.items):
                        if self.BOM[j][i] > 0:
                            self.allDemand[j][t] += self.BOM[j][i]*self.c_extDemand[i][t]
                            self.c_averageDemand[j] += self.BOM[j][i]*self.c_extDemand[i][t]
                self.c_averageDemand[i] /= self.periods
            self.averagePeriodDemand = 0.0                              # compute average period demand (in capacity units)
            for i in range(self.items):
                for t in range(self.periods):
                    self.averagePeriodDemand += self.allDemand[i][t]
            self.averagePeriodDemand /= self.periods
            self.TBO = np.zeros(self.items, dtype=float)                # compute TBO and average TBO
            self.c_TBO = self.TBO
            averageSetupCap = 0
            for i in range(self.items):
                self.c_TBO[i] = sqrt((2*self.c_setupCost[i])/(self.c_holdingCost[i]*self.averageDemand[i]))
                averageSetupCap += self.c_setupTime[self.itemmachine[i]][i] / self.c_TBO[i]
            self.averageTBO = self.TBO.mean()
            
cdef class MLCLSP(MLCLSP_data):
    """General MLCLSP class (derived from MLCLSP_data).
    It loads data for instance file and generates solution data fields.
    
    Data and variables:
      instance related data (from MLCLSP_data):
        instanceName (string): name of the loaded instance
        items (int): number of items
        periods (int): number of periods
        machines (int): number of machines
        demand (numpy.array, dimension: items,periods): demand for each item and period  (only external demand on the current planning level)
        capacity (numpy.array, dimension: machines, periods): capacities for each machine for each period
        initInventory (numpy.array, dimension: items): initial inventory for each item
        prodCoeff (numpy.array, dimension: items): capacity consumption for each item
        holdingCost (numpy.array, dimension: items): holding cost for each item (per unit, per period)
        setupCost (numpy.array, dimension: items): setup cost for each item
        setupTime (numpy.array, dimension: machines, items): capacity consumption of setup operation for each item
        overtimeCost (double): overtime cost per unit capacity (set to 10000, not loaded from instance file)

      production plan related data:
        lotsizes (numpy.array (double), dimension: items,periods): lot sizes (production plan)
        setup (numpy.array (bool), dimension: items,periods): setup decisions
        inventory (numpy.array (double), dimension: items,periods): inventory levels at the end of each period
        origAvCap (numpy.array (double), dimension: machines,periods): original available capacity in production plan
        totalHoldingCost (double): total holding cost of production plan
        totalSetupCost (double): total setup cost of production plan
        totalOvertimeCost (double): total overtime cost of production plan
        totalCost (double): total cost of production plan
        
        ..._store: storage containers for the variables for storing solutions (first index gives the number of the storage container)
        
        c_... C memoryviews of numpy arrays and Python variables
      
      2-SCH handling variables:
        cost_amounts (numpy.array, dimensions: periods): lot sizes (incremental) for item i reported after checking EXT and NEW options
        postponementDL (DemandList): postponement demand list
        desactivatePostponement (bool): postponement routine must be desactivated in case I when we use inventory
        postpone (bool): demand element should be put in the postponement demand list (=1) or not (=0)
        DL (bool): work with main demand list (=0) or postponement demand list (=1)
        usePostDL (bool): postponement demand list must be checked (=1) or not (=0)
        notAllNow (bool): indicates whether it is possible to add the whole demand element to its period (=0) or not (=1)
        keepInPostDL (bool): demand element from postponement demand list is postponed again (=1) or not (=0)
        nextPos (int): the position after the last element in the postponement demand list
        
        avCap (numpy.array (double), dimension: machines, periods): adapted available capacity in production plan
        capReq (numpy.array (double), dimension: items, machines): capacity requirement for each item on each machine (accounts for the requirements of all predecessors of an item)
        firstPeriodPlanned (numpy.array (double), dimension: items): demand in the first period is planned from the start for all levels
        projDemand (numpy.array (double), dimension: items, periods): projected demand for items on the next level based on lots of the current level
        blockCap (numpy.array, dimension: machines, periods): capacity for each machine for each period needed to accommodate the production of the predecessors of the currenly scheduled items
        thislevelitems(list): items that are to be planned on the current planning level
        
      2-SCH versions control:
        shiftProduction (bool): shift of production routine is enabled (=1) or not (=0)
        postponemnt (bool): postponement routine is enabled (=1) or not (=0)
        threshold (double): threshold value
      
    Methods:
        __init__(path, filename, storage): initialize the class and load the data from the given instance file. The demand of the first period is planned.
        __str__(): convert solution into string for output.
        resetSolution(): set the production plan to the initial (actual plan of the first period demand).
        storeSolution(container): store current solution in container (container=0 is reserved for resetSolution-function)
        retrieveSolution(container): retrieve solution from container
        update_totalHoldingCost(): update the total holding cost and return it
        update_totalSetupCost(): update the total setup cost and return it
        update_totalOvertimeCost(): update the total overtime cost and return it
        update_totalCost(): update the total cost and return it
        reset_postponementDL(): empty postponement demand list
        try_addLot2Period(i, t, tt, amount, maxamount, m, dummyCost): try to add amount units of item i, used to satisfy demand in period t, to the production on machine m in period tt
        addLot2Period(i, t, tt, amount, m): actually add amount units of item i, used to satisy demand in period t, to the production on machine m in period tt
        _useInv ( i, t, amount, m): case I of 2-SCH - use existing inventory to satisfy (partially) demand of amount units of item i on machine m in t
        try_EXTandNEW(i, t, amount, m): options EXT and NEW - compute incremental cost of extending existing lots (EXT) and/or creating a new lot (NEW) to satisfy demand of amount units of item i on machine m in period t
        add_EXTorNEW(i, t, m): execute options EXT / NEW for demand of item i on machine m in period t
        postponeDemand(i, t, amount, firstPostponement): put demand element of amount units of item i in period t to the postponement demand list
        _useOvertime(i, t, amount, m): case IV of 2-SCH - use overtime to satisfy demand of amount units of item i on machine m in t
        addDemand(i, t, m, amount, firstPostponement): add demand of amount units of item i in period t to the production plan, firstPostponement indicates whether this demand element is already in postponement list or not yet
        sortItems(criterion): demand elements sorted according to an item-based criterion
        sortPeriods(criterion): demand elements sorted according to a period-based criterion
        generateDL(criterionPeriod, criterionItem, choice): generate demand list with period-based and cost-based criteria, choice indicates which criterion is the primary key
        generateDLdemand(criterionPeriod, criterionDemand): generate demand list with period-based and capacity-based criteria
        update_blockedCap(): update blocked capacity on all machines in all periods for not yet scheduled predecessors of items of the current level
        reset_projDemand(): set projected demand to 0
        update_projDemand(t, i, amount): update projected demand for all predecessors of item i added to period t
        update_projDemandShift(t_from, t_to, i, amount): update projected demand for all predecessors of item i when its lot is shifted from period t_from to period t_to
        checkCapacity(i, m, t, amount): check available capacity on machine m in period t, taking the blocked capacity into account
        update_extDemand(lotsizes): update external demand for the current planning level
        planDemandList(dl, resetSol, pos1, pos2, resetPostDL): create a production plan using demand list dl for demand elements between pos1 and pos2, resetSol indicates whether should start from empty production plan, resetPostDL - whether postponement demand list must be emptied.
        """
    
    # definition of class variables
    cdef public np.ndarray lotsizes, setup, inventory, avCap, cost_amounts, origAvCap, capReq, firstPeriodPlanned, projDemand, blockedCap
    cdef public double totalHoldingCost, totalSetupCost, totalOvertimeCost, totalCost
    cdef double[:,:] c_lotsizes
    cdef uint8[:,:] c_setup
    cdef double[:,:] c_inventory
    cdef double[:,:] c_origAvCap
    cdef double[:,:] c_avCap
    cdef double[:] c_cost_amounts
    cdef double[:,:] c_capReq
    cdef double [:] c_firstPeriodPlanned
    cdef double[:,:] c_projDemand
    cdef double[:,:] c_blockedCap
    
    # storage containers for the results
    cdef np.ndarray lotsizes_store, setup_store, inventory_store, origAvCap_store
    cdef np.ndarray totalHoldingCost_store, totalSetupCost_store, totalOvertimeCost_store, totalCost_store
    cdef double[:,:,:] c_lotsizes_store
    cdef uint8[:,:,:] c_setup_store
    cdef double[:,:,:] c_inventory_store
    cdef double[:,:,:] c_origAvCap_store
    cdef double[:] c_totalHoldingCost_store
    cdef double[:] c_totalSetupCost_store
    cdef double[:] c_totalOvertimeCost_store
    cdef double[:] c_totalCost_store
    
    #parameters for 2-SCH
    cdef uint8 shiftProduction, postponement
    cdef double threshold
    
    cdef uint8 desactivatePostponement, postpone, DL, usePostDL, notAllNow, keepInPostDL
    cdef int nextPos
    cdef DemandList postponementDL
    cdef public object thislevelitems
    
    # constructor - calls CLSP_data.__init__, storage gives the number of initialized storage containers
    def __init__(self, str path, str filename, uint8 setuptimezero = False, int storage = 1,
                 uint8 shiftProduction = True, uint8 postponement = True, double threshold = 0.5):
        # define 2-SCH version
        self.shiftProduction = shiftProduction
        self.postponement = postponement
        self.threshold = threshold
        
        # instance and production plan related data
        MLCLSP_data.__init__ (self, path, filename, setuptimezero)
        self.lotsizes = np.zeros((self.items,self.periods), dtype=float, order="C")
        self.c_lotsizes = self.lotsizes
        self.setup = np.zeros((self.items,self.periods),dtype=np.uint8,order="C")
        self.c_setup = self.setup
        self.inventory = np.zeros((self.items,self.periods), dtype=float, order="C")
        self.c_inventory = self.inventory
        self.avCap = np.zeros((self.machines,self.periods),dtype=float)
        self.c_avCap = self.avCap
        self.c_avCap[...] = self.c_capacity
        self.blockedCap = np.zeros((self.machines,self.periods),dtype=float)
        self.c_blockedCap = self.blockedCap
        self.projDemand = np.zeros((self.items,self.periods),dtype=float)
        self.c_projDemand = self.projDemand
        
        # extra storage data
        self.lotsizes_store = np.zeros((storage+1,self.items,self.periods), dtype=float, order="C")
        self.c_lotsizes_store = self.lotsizes_store
        self.setup_store = np.zeros((storage+1,self.items,self.periods),dtype=np.uint8,order="C")
        self.c_setup_store = self.setup_store
        self.inventory_store = np.zeros((storage+1,self.items,self.periods), dtype=float, order="C")
        self.c_inventory_store = self.inventory_store
        self.origAvCap_store = np.zeros((storage+1,self.machines,self.periods),dtype=float, order="C")
        self.c_origAvCap_store = self.origAvCap_store
        self.totalHoldingCost_store = np.zeros(storage+1,dtype=float)
        self.c_totalHoldingCost_store = self.totalHoldingCost_store
        self.totalSetupCost_store = np.zeros(storage+1,dtype=float)
        self.c_totalSetupCost_store = self.totalSetupCost_store
        self.totalOvertimeCost_store = np.zeros(storage+1,dtype=float)
        self.c_totalOvertimeCost_store = self.totalOvertimeCost_store
        self.totalCost_store = np.zeros(storage+1,dtype=float)
        self.c_totalCost_store = self.totalCost_store
        
        # initialize handling variables
        self.cost_amounts = np.zeros(self.periods, dtype=float)
        self.c_cost_amounts = self.cost_amounts
        self.postpone = 0
        self.usePostDL = 1
        self.DL = 0
        self.nextPos = 0
        self.keepInPostDL = False
        self.notAllNow = False
        self.desactivatePostponement = False
        self.thislevelitems = []
        
        # initialize postponement demand list
        self.postponementDL = DemandList(self.nonzerodemand)
        
        # plan demand in the first period and determine the cost of production plan
        self.totalHoldingCost = 0.0
        self.totalSetupCost = 0.0
        for i in range(self.items):
            m = self.c_itemmachine[i]
            if (self.c_extDemand[i][0] > MyAbsTol):
                self.c_lotsizes[i][0] = self.c_extDemand[i][0] - self.initInventory[i]
                self.thislevelitems.append(i)
            for j in range(self.items):
                if (self.c_gozinto[i][j] > MyAbsTol) and (self.c_lotsizes[j][0] > MyAbsTol):
                    self.c_lotsizes[i][0] += self.c_gozinto[i][j]*self.c_lotsizes[j][0] - self.initInventory[i]   
            if self.c_lotsizes[i][0] > MyAbsTol:
                self.c_avCap[m][0] -= self.c_lotsizes[i][0] + self.c_setupTime[m][i]
                self.c_setup[i][0] = True
                self.totalSetupCost += self.c_setupCost[i]
        self.firstPeriodPlanned = np.zeros(self.items, dtype=float, order='C')
        self.c_firstPeriodPlanned = self.firstPeriodPlanned
        self.c_firstPeriodPlanned[...] = self.c_lotsizes[:, 0]
        for m in range(self.machines):
            if self.c_avCap[m][0] < -MyAbsTol:
                self.totalOvertimeCost = self.overtimeCost * (-self.c_avCap[m][0])
            else:
                self.totalOvertimeCost = 0.0
                self.origAvCap = np.zeros((self.machines,self.periods),dtype=float)
        self.c_origAvCap = self.origAvCap
        self.c_origAvCap[...] = self.c_avCap
        self.totalCost = self.totalSetupCost + self.totalOvertimeCost
        #compute capacity requirement of predecessors for each machine and each item for 1 unit
        self.capReq = np.zeros((self.items, self.machines), dtype=float, order='C')
        self.c_capReq = self.capReq
        for i in range(self.items):
            for j in range(self.items):
                if self.gozinto[j][i] > 0:
                    m = self.itemmachine[j]
                    self.c_capReq[i][m] += self.gozinto[j][i]
                    while np.sum(self.gozinto[:,j])>0:
                        for jj in range(self.items):
                            m = self.itemmachine[jj]
                            self.c_capReq[i][m] += self.gozinto[jj][j]*self.gozinto[j][i]
                            j = jj
        self.storeSolution(0)
    
    # create a string with production plan data 
    def __str__(self):
        ostring = self.instanceName + "\n"
        ostring += "Cost: " + str(self.totalCost) + "(S: " + str(self.totalSetupCost) + ", H: " + str(self.totalHoldingCost) + ", O: " + str(self.totalOvertimeCost) + ")\n"
        ostring += "Production plan: \n" + str(self.lotsizes) + "\n"
        ostring += "Inventory: \n" + str(self.inventory) + "\n"
        ostring += "Available capacity: \n" + str(self.avCap) + "\n"
        return ostring
    
    # reset production plan to initial
    cpdef void resetSolution(self):
        self.retrieveSolution(0)
    
    # store solution in container (container=0 is reserved)
    cpdef void storeSolution(self, int container = 1):
        self.c_lotsizes_store[container][...] = self.c_lotsizes
        self.c_setup_store[container][...] = self.c_setup
        self.c_inventory_store[container][...] = self.c_inventory
        self.c_origAvCap_store[container][...] = self.c_origAvCap
        self.c_totalHoldingCost_store[container] = self.totalHoldingCost
        self.c_totalSetupCost_store[container] = self.totalSetupCost
        self.c_totalOvertimeCost_store[container] = self.totalOvertimeCost
        self.c_totalCost_store[container] = self.totalCost
    
    # retrieve solution in container (container=0 is reserved)
    cpdef void retrieveSolution(self, int container = 1):
        self.c_lotsizes[...] = self.c_lotsizes_store[container]
        self.c_setup[...] = self.c_setup_store[container]
        self.c_inventory[...] = self.c_inventory_store[container]
        self.c_origAvCap[...] = self.c_origAvCap_store[container]
        self.totalHoldingCost = self.c_totalHoldingCost_store[container]
        self.totalSetupCost = self.c_totalSetupCost_store[container]
        self.totalOvertimeCost = self.c_totalOvertimeCost_store[container]
        self.totalCost = self.c_totalCost_store[container]

    # recalculate the holding cost
    cdef double update_totalHoldingCost(self):
        self.totalHoldingCost = 0
        cdef int i,p
        cdef double sumInvP
        for i in range(self.items):
            sumInvP = 0
            for p in range(self.periods):
                sumInvP += self.c_inventory[i][p]
            self.totalHoldingCost += sumInvP * self.c_holdingCost[i]
        return self.totalHoldingCost

    # recalculate setup cost
    cdef double update_totalSetupCost(self):
        self.totalSetupCost = 0
        cdef int i,p
        cdef double sumSetupP
        for i in range(self.items):
            sumSetupP = 0
            for p in range(self.periods):
                sumSetupP += self.c_setup[i][p]
            self.totalSetupCost += sumSetupP * self.c_setupCost[i]
        return self.totalSetupCost

    # recalculate overtime cost
    cdef double update_totalOvertimeCost(self):
        self.totalOvertimeCost = 0
        cdef int p
        for p in range(self.periods):
            for m in range(self.machines):
                if self.c_avCap[m][p] < -MyAbsTol:
                    self.totalOvertimeCost -= self.c_avCap[m][p]
        self.totalOvertimeCost *= self.overtimeCost
        return self.totalOvertimeCost

    # recalculate total cost
    cpdef double update_totalCost(self):
        self.totalCost = self.update_totalHoldingCost() + self.update_totalSetupCost() + self.update_totalOvertimeCost()
        return self.totalCost
    
    # reset postponement demand list
    cpdef void reset_postponementDL(self):
        self.postponementDL = DemandList(self.nonzerodemand)
    
    # try to add a lot (demand for period t) to a specific period (tt)
    # returns: cost (incremental); maxamount (maximum amount that can be added) is returned via pointer
    cdef double try_addLot2Period(self, int i, int t, int tt, double amount, double* maxamount, int m, uint8 dummyCost=False):
        
        cdef uint8 activeShift = False # control if shift is enabled
        
        if amount <= MyAbsTol:
            maxamount[0] = 0
            return 0
        
        cdef double scost = 0     # setup cost (incremental)
        
        if self.c_setup[i][tt] == False or self.shiftProduction == True:
            activeShift = True
        
        # compute the amount that can be added 
        maxamount[0] = self.checkCapacity(i, m, tt, amount)
        if not self.c_setup[i][tt]:
            scost = self.c_setupCost[i]    # if there was no setup, there is additional setup cost
        
        if maxamount[0] <= MyAbsTol:       # no production can be added to period tt
            maxamount[0] = 0
            return -1
        
        #local variables to handle shifts
        cdef double cost_sav_inv = 0
        cdef double add_cap_inv = 0
        cdef int tt_back
        cdef double maxfrominv
        cdef double cost_sav_future = 0
        cdef double avcap_future = 0
        cdef int tt_future = tt + 1
        
        if activeShift:
            # right shift
            if tt>0:
                if self.c_inventory[i][tt-1] > MyAbsTol:
                    maxfrominv = fmin(self.c_inventory[i][tt-1],self.c_avCap[m][tt]-maxamount[0])
                    add_cap_inv = maxfrominv   
                    if maxfrominv > MyAbsTol:
                        tt_back = tt-1
                        while maxfrominv > MyAbsTol:
                            while self.c_setup[i][tt_back] == False:
                                cost_sav_inv += self.c_holdingCost[i] * maxfrominv
                                tt_back -= 1
                            cost_sav_inv += self.c_holdingCost[i] * maxfrominv
                            if (maxfrominv - self.c_lotsizes[i][tt_back])>= -MyAbsTol:
                                maxfrominv -= self.c_lotsizes[i][tt_back]
                                cost_sav_inv += self.c_setupCost[i]
                            else:
                                break
                            tt_back -= 1
            
            # left shift
            avcap_future = self.c_avCap[m][tt]-maxamount[0]-add_cap_inv
            self.update_projDemand(tt, i, maxamount[0]+add_cap_inv)
            while (avcap_future > MyAbsTol and tt_future < self.periods):
                if self.c_setup[i][tt_future]==True:
                    amount_future = self.c_lotsizes[i][tt_future]
                    if dummyCost:
                        amount_future += self.c_cost_amounts[tt_future]
                    if amount_future <= self.checkCapacity(i, m, tt, amount_future):
                        if amount_future*self.c_holdingCost[i]*(tt_future-tt) < self.c_setupCost[i]:
                            avcap_future -= amount_future
                            cost_sav_future += self.c_setupCost[i] - amount_future*self.c_holdingCost[i]*(tt_future-tt)
                    else:
                        break
                tt_future += 1
            self.update_projDemand(tt, i, -(maxamount[0]+add_cap_inv))
        
        return scost + maxamount[0]*self.c_holdingCost[i]*(t-tt) - cost_sav_inv - cost_sav_future
    
    # adds the amount (demand for period t) to production in period tt
    # returns total cost of current plan
    cdef double addLot2Period(self, int i, int t, int tt, double amount, int m):
        cdef uint8 activeShift = False

        if amount <= MyAbsTol:
            return self.totalCost
        
        self.c_lotsizes[i][tt] += amount
        
        cdef double avCapprv = self.c_avCap[m][tt]
        self.c_avCap[m][tt] -= amount
        
        cdef int p
        for p in range(tt,t):
            self.c_inventory[i][p] += amount
        self.totalHoldingCost += (t-tt) * self.c_holdingCost[i] * amount
        if self.c_setup[i][tt] == False or self.shiftProduction == True:
            activeShift = True
        if self.c_setup[i][tt] == False:
            self.c_setup[i][tt] = True
            self.c_avCap[m][tt] -= self.c_setupTime[m][i]
            self.totalSetupCost += self.c_setupCost[i]
            
        if self.c_avCap[m][tt] < -MyAbsTol:
            self.totalOvertimeCost += (-self.c_avCap[m][tt])*self.overtimeCost - (fmax(-avCapprv,0)*self.overtimeCost)
        self.totalCost = self.totalHoldingCost + self.totalSetupCost + self.totalOvertimeCost

        self.update_projDemand(tt, i, amount)
        
        cdef double maxfrominv
        cdef int tt_back

        cdef int tt_future = tt + 1
        cdef int ttt
        
        if activeShift:
            # right shift
            if tt>0:
                if self.c_inventory[i][tt-1] > MyAbsTol:
                    maxfrominv = fmin(self.c_inventory[i][tt-1],self.c_avCap[m][tt])
                    if maxfrominv > MyAbsTol:
                        self.c_lotsizes[i][tt] += maxfrominv
                        self.c_avCap[m][tt] -= maxfrominv
                        tt_back = tt-1
                        while maxfrominv > MyAbsTol:
                            while self.c_setup[i][tt_back] == False:
                                self.c_inventory[i][tt_back] -= maxfrominv
                                self.totalHoldingCost -= self.c_holdingCost[i] * maxfrominv
                                tt_back -= 1
                            self.c_inventory[i][tt_back] -= maxfrominv
                            self.totalHoldingCost -= self.c_holdingCost[i] * maxfrominv
                            if (maxfrominv - self.c_lotsizes[i][tt_back])>= -MyAbsTol:
                                self.update_projDemandShift(tt_back, tt, i, self.c_lotsizes[i][tt_back])
                                maxfrominv -= self.c_lotsizes[i][tt_back]
                                self.c_setup[i][tt_back] = False
                                self.totalSetupCost -= self.c_setupCost[i]
                                self.c_avCap[m][tt_back] += self.c_lotsizes[i][tt_back] + self.c_setupTime[m][i]  
                                self.c_lotsizes[i][tt_back] = 0
                            else:
                                self.c_avCap[m][tt_back] += maxfrominv
                                self.update_projDemandShift(tt_back, tt, i, maxfrominv)
                                self.c_lotsizes[i][tt_back] -= maxfrominv
                                break
                            tt_back -= 1

            # left shift
            while (self.c_avCap[m][tt] > MyAbsTol and tt_future < self.periods):
                if self.c_setup[i][tt_future]==True:
                    if self.c_lotsizes[i][tt_future] <= self.checkCapacity(i, m, tt, self.c_lotsizes[i][tt_future]):
                        if self.c_lotsizes[i][tt_future]*self.c_holdingCost[i]*(tt_future-tt) < self.c_setupCost[i]:
                            self.c_avCap[m][tt] -= self.c_lotsizes[i][tt_future]
                            self.c_lotsizes[i][tt] += self.c_lotsizes[i][tt_future]
                            self.update_projDemandShift(tt_future, tt, i, self.c_lotsizes[i][tt_future])
                            self.c_setup[i][tt_future] = False
                            self.totalSetupCost -= self.c_setupCost[i]
                            for ttt in range(tt,tt_future):
                                self.c_inventory[i][ttt] += self.c_lotsizes[i][tt_future]
                            self.totalHoldingCost += self.c_lotsizes[i][tt_future]*self.c_holdingCost[i]*(tt_future-tt)
                            self.c_avCap[m][tt_future] += self.c_lotsizes[i][tt_future] + self.c_setupTime[m][i]   
                            self.c_lotsizes[i][tt_future] = 0.0
                        else:
                            break
                tt_future += 1
        
        self.update_totalCost()

        return self.totalCost
    
    # case I of 2-SCH: use existing inventory to satisfy (partially) demand
    # returns remaining amount to be scheduled
    cdef double _useInv (self, int i, int t, double amount, int m):
        cdef int p
        cdef double avCap = 0
        cdef double possible_amount = 0
        cdef double remaining_amount = 0
        if self.c_inventory[i][t] > MyAbsTol:     # if there is positive inventory
            for p in range(t+1,self.periods):
                avCap += fmax(self.c_avCap[m][p],0)
            if avCap > MyAbsTol:                  # and if there is available capacity for production after period t
                possible_amount = fmin(fmin(self.c_inventory[i][t], amount),avCap) # compute what is the amount that can be satisfied from inventory: = d^new
                remaining_amount = amount - possible_amount # compute the remaining amount: =d^rem
                self.c_inventory[i][t] -= possible_amount
                self.totalHoldingCost -= self.c_holdingCost[i] * possible_amount
                self.totalCost -= self.c_holdingCost[i] * possible_amount
                self.desactivatePostponement = True
                self.addDemand(i, t + 1, m, possible_amount, True) # add the "new demand element" in the next period immediately
                return remaining_amount
        return amount
            
    # options EXT and NEW: compute backwards cost of extending existing lots and/or creating new lots
    # returns cost (incremental) of the cheapest option
    cdef double try_EXTandNEW(self, int i, int t, double amount, int m):
        cdef uint8 NoSetupPeriodFound = False
        cdef double costEXT = 0
        cdef double costNEW = 1e+24, thiscost = 1e+24
        cdef int tt = t-1
        cdef int ttt = t
        cdef double amount2 = 0, allowed = 0
        cdef double possible_amount_rem = amount
        cdef double possible_amount_3 = 0
        cdef double[:,:] c_projDemand_init=np.zeros((self.items, self.periods))
        c_projDemand_init[...] = self.c_projDemand
        self.c_cost_amounts[...] = 0
        self.postpone = 0

        while possible_amount_rem > MyAbsTol and tt >= 0:
            costEXT += possible_amount_rem * self.c_holdingCost[i]
            amount2 = self.c_avCap[m][tt]
            if self.c_setup[i][tt] and amount2 > MyAbsTol:
                possible_amount_rem -= amount2
                self.c_cost_amounts[tt] = min(possible_amount_rem+amount2, amount2)
                allowed = self.checkCapacity(i, m, tt, self.c_cost_amounts[tt])
                if allowed < self.cost_amounts[tt]:
                    possible_amount_rem += amount2
                    possible_amount_rem -= max(0, allowed)
                    self.cost_amounts[tt] = allowed
                self.c_projDemand[i][tt] += self.cost_amounts[tt]
                self.update_projDemand(tt, i, self.cost_amounts[tt])
            if not self.c_setup[i][tt]:
                amount2 -= self.c_setupTime[m][i]       
                if amount2 >= possible_amount_rem:
                    if not NoSetupPeriodFound:
                        thiscost = self.try_addLot2Period(i, t, tt, possible_amount_rem, &possible_amount_3, m, dummyCost=True)
                        if possible_amount_3 >= possible_amount_rem:
                            if thiscost < costNEW:
                                costNEW = thiscost
                                self.c_cost_amounts[tt] = possible_amount_rem
                                self.c_projDemand[i][tt] += self.cost_amounts[tt]
                                self.update_projDemand(tt, i, self.cost_amounts[tt])
                                ttt = tt
                                NoSetupPeriodFound = True
            tt -= 1
            if costEXT > costNEW:
                break
        
        if possible_amount_rem > MyAbsTol:
            costEXT = 1e+24
            
        self.c_projDemand[...] = c_projDemand_init
        self.update_blockedCap()
        
        if self.postponement and (costEXT != 1e+24):
            if (abs(costEXT-costNEW) <= self.threshold * costEXT) and (self.DL == 0) and not self.desactivatePostponement:
                self.postpone = 1
        
        if costEXT <= costNEW:
            self.c_cost_amounts[ttt] = 0
            return costEXT
        else:
            self.c_cost_amounts[:ttt] = 0
            return costNEW
            
    # execute the cheapest option from EXT and NEW
    cdef double add_EXTorNEW(self, int i, int t, int m):
        cdef int tt = t-1
        
        while tt >= 0:
            if tt == 0:
                return self.addLot2Period(i, t, tt, self.c_cost_amounts[tt], m)
            self.addLot2Period(i, t, tt, self.c_cost_amounts[tt], m)
            tt -= 1
    
    # put demand element to the postponement demand list
    cdef double postponeDemand(self, int i, int t, double amount, firstPostponement=True):
        cdef int pos, period
        
        if firstPostponement:
            self.postponementDL.c_item_index[self.nextPos] = i
            self.postponementDL.c_period_index[self.nextPos] = t
            self.postponementDL.c_amounts[self.nextPos] = amount
            self.nextPos += 1
        else:
            for pos in range(0, self.nextPos):
                if i == self.postponementDL.c_item_index[pos] and t == self.postponementDL.c_period_index[pos]:
                    self.postponementDL.c_amounts[pos] = amount
            self.keepInPostDL = True
        self.usePostDL = 0
        return self.totalCost
    
    # case IV of 2-SCH: use overtime
    # returns total cost of current plan
    cdef double _useOvertime(self, int i, int t, double amount, int m):
        cdef int tt = t-1
        cdef int t_inv = tt
        cdef double amount2, allowed

        while tt >= 0: # fill available capacity with production in earlier periods as much as possible
            if (self.c_avCap[m][tt] - (self.c_setupTime[m][i]*(1-self.c_setup[i][tt]))) > MyAbsTol:
                amount2 = min(amount, (self.c_avCap[m][tt]- (self.c_setupTime[m][i]*(1-self.c_setup[i][tt]))))
                allowed = self.checkCapacity(i, m, tt, amount2)
                if allowed < amount2:
                    amount2 = allowed
                if amount2 > MyAbsTol:
                    if self.c_setup[i][tt] == False:
                        self.c_setup[i][tt] = True
                        self.totalSetupCost += self.c_setupCost[i]
                        self.c_avCap[m][tt] -= self.c_setupTime[m][i] 
                    self.c_lotsizes[i][tt] += amount2
                    self.c_avCap[m][tt] -= amount2 
                    for t_inv in range(tt, t):
                        self.c_inventory[i][t_inv] += amount2
                        self.totalHoldingCost += self.c_holdingCost[i]*amount2*(t-t_inv)
                    self.update_projDemand(tt, i, amount2)
                    amount -= amount2
            tt -= 1
        if amount > MyAbsTol: # put the rest as overtime in period t
            if self.c_setup[i][t] == False:
                self.c_setup[i][t] = True
                self.totalSetupCost += self.c_setupCost[i]
                self.c_avCap[m][t] -= self.c_setupTime[m][i]   
            self.c_lotsizes[i][t] += amount
            self.c_avCap[m][t] -= amount
            self.totalOvertimeCost += self.overtimeCost*max(0, -self.c_avCap[m][t])
            self.update_projDemand(t, i, amount)
        
        self.update_totalCost()
        
        return self.totalCost
    
    # extend partial production plan: manages all cases of 2-SCH for a demand element
    # returns total cost
    cdef double addDemand(self, int i, int t, int m, double amount, uint8 activeUseInv=True, uint8 firstPostponement=True):
        cdef double possible_amount = 0
        cdef double remaining_amount = 0
        cdef double cost = 0
        cdef double other_cost = 0
                
        self.notAllNow = False

        if amount <= MyAbsTol:          # check if amount is positive, otherwise no change of the production plan
            return self.totalCost
        
        # case I: Use inventory
        if (activeUseInv and (t < self.periods-1)): 
            remaining_amount = self._useInv(i,t,amount,m)
            if remaining_amount <= MyAbsTol:
                return self.totalCost
            self.notAllNow = False
            amount = remaining_amount
        
        # if we are in first period (t=0), we have to add the amount here
        if t == 0:                  
            return self.addLot2Period(i, t, t, amount, m)
        
        cost = self.try_addLot2Period(i,t,t,amount,&possible_amount,m)    # compute option NOW
        remaining_amount = amount-possible_amount
        if possible_amount <= MyAbsTol:
            self.notAllNow = True
        
        # case II: enough capacity, everything can be added to t
        if remaining_amount <= MyAbsTol:  # if everything can be added but with additional cost (setup)
            other_cost = self.try_EXTandNEW(i, t, amount, m)
            if (cost <= other_cost) and not self.notAllNow: #option NOW - cheaper to add everything to t (either create a new lot or extend the existing lot)
                return self.addLot2Period(i, t, t, amount, m)
            elif (self.DL == 0) and (self.postpone == 1): # POSTPONEMENT routine - cheaper to create/extend previous lot/s but the difference between both is too small, so postpone 
                return self.postponeDemand(i, t, amount, firstPostponement)
            else:
                return self.add_EXTorNEW(i, t, m) # execute option EXT or NEW
        
        # case III: not enough capacity, only part or nothing can be added to t
        
        # if there are no additional cost, but not the whole amount can be added
        if cost <= 0 and not self.notAllNow:           
            self.addLot2Period(i,t,t,possible_amount,m)   # add what is possible
            other_cost = self.try_EXTandNEW(i, t, remaining_amount, m) # try options EXT and NEW for the remaining amount (translates to options NOW+EXT and NOW+NEW options in case III)
            if other_cost == 1e+24:                     # if options EXT and NEW are infeasible (=1e+24)
                return self._useOvertime(i, t, remaining_amount, m) # case IV: use overtime
            if (self.DL == 0) and (self.postpone == 1): # POSTPONEMENT routine - options EXT and NEW are feasible but too small difference, so postpone
                    return self.postponeDemand(i, t, remaining_amount, firstPostponement)
            else:                                       
                return self.add_EXTorNEW(i, t, m) # execute option EXT or NEW
        
        # if there is additional cost and not everything can be added to t
        else:                                           
            self.c_projDemand[i][t] += possible_amount
            self.update_projDemand(t, i, possible_amount)
            other_cost2 = self.try_EXTandNEW(i, t, remaining_amount, m) # try options EXT and NEW for the remaining amount (translates to options NOW+EXT and NOW+NEW options in case III)
            self.c_projDemand[i][t] -= possible_amount
            self.update_projDemand(t, i, -possible_amount)
            other_cost = self.try_EXTandNEW(i, t, amount, m)            # try options EXT and NEW for the whole amount
            if (other_cost < other_cost2+cost) and (other_cost2 != 1e+24): # EXT or NEW is cheaper
                other_cost = self.try_EXTandNEW(i, t, amount, m)
            if other_cost2 == 1e+24:                    # if options EXT and NEW are infeasible (=1e+24)
                return self._useOvertime(i, t, amount, m)  # case IV: use overtime
            if (cost+other_cost2 <= other_cost) and not self.notAllNow: # NOW+EXT or NOW+NEW is cheaper
                self.addLot2Period(i,t,t,possible_amount, m)   # add what is possible
                other_cost2 = self.try_EXTandNEW(i, t, remaining_amount, m)
                if (self.DL == 0) and (self.postpone == 1): # POSTPONEMENT routine - options EXT and NEW are feasible but too small difference, so postpone
                    return self.postponeDemand(i, t, remaining_amount, firstPostponement)
                else:
                    return self.add_EXTorNEW(i, t, m)          # execute option EXT or NEW
            elif (self.DL == 0) and (self.postpone == 1):
                return self.postponeDemand(i, t, amount, firstPostponement)
            else:
                return self.add_EXTorNEW(i, t, m)

            
    # sort items according to some criteria
    # returns a sorted list of item indices
    def sortItems(self, str criterion):
        itemlist = []
        for it in range(0, self.items):
            itemlist.append(it)
        if criterion == "SH":
            itemlist.sort(key=lambda x: (self.setupCost[x]/self.holdingCost[x]),reverse=True)
        elif criterion == "HS":
            itemlist.sort(key=lambda x: (self.holdingCost[x]/self.setupCost[x]), reverse=True)
        elif criterion == "TBO":
            itemlist.sort(key=lambda x: (self.TBO[x]), reverse=True)
        elif criterion == "HSC":
            itemlist.sort(key=lambda x: (self.holdingCost[x]/(self.setupCost[x]*self.averageDemand[x])), reverse=True)
        elif criterion == "EC":
            itemlist.sort(key=lambda x: ((self.setupCost[x]/self.TBO[x])+(self.TBO[x]*self.holdingCost[x]*self.averageDemand[x]/2)), reverse=True)
        elif criterion == "ES":
            itemlist.sort(key=lambda x: ((self.TBO[x]-1)*self.setupCost[x]-(self.TBO[x]*(self.TBO[x]-1)*self.holdingCost[x]*self.averageDemand[x]/2)), reverse=True)
        elif criterion == "ESC":
            itemlist.sort(key=lambda x: ((self.TBO[x]-1)*self.setupCost[x]-(self.TBO[x]*(self.TBO[x]-1)*self.holdingCost[x]*self.averageDemand[x]/2))/self.averageDemand[x], reverse=True)
        elif criterion == "CUI":
            itemlist.sort(key=lambda x: (self.extDemand[x, :]).sum(), reverse=True)
        elif criterion == "S":
            itemlist.sort(key=lambda x: self.setupCost[x], reverse=True)
        elif criterion == "H":
            itemlist.sort(key=lambda x: self.holdingCost[x], reverse=True)
        else:
            print("Invalid Criterion!")
        return itemlist

    # sort items according to some criteria
    # returns a sorted list of period indices
    def sortPeriods(self, str criterion, FirstPeriod=False):
        periodlist = []
        if FirstPeriod:
            for p in range(0, self.periods):
                periodlist.append(p)
        else:
            for p in range(1, self.periods):
                periodlist.append(p)
        if criterion == "INCP":
            return periodlist
        elif criterion == "DECP":
            periodlist.sort(key=lambda x: x, reverse=True)
        elif criterion == "CUP":
            periodlist.sort(key=lambda x: (self.extDemand[:, x]).sum(), reverse=True)
        else:
            print("Invalid Criterion!")
        return periodlist

    # generate demand list with specific sorting critieria (only for cost-based criteria)
    # returns demand list
    def generateDL(self, str criterionPeriod, str criterionItem, str choice, uint8 FirstPeriod=False):
        itemList = self.sortItems(criterionItem)                  
        periodList = self.sortPeriods(criterionPeriod, FirstPeriod)           
        dl = DemandList(self.nonzerodemand)
        cdef int pos = 0
        cdef int i, t
        if choice == "PI":
            for t in periodList:
                for i in itemList:
                    if self.c_extDemand[i][t] > MyAbsTol:
                        dl.c_item_index[pos] = i
                        dl.c_period_index[pos] = t
                        dl.c_amounts[pos] = self.c_extDemand[i][t]
                        pos += 1
        elif choice == "IP":
            for i in itemList:
                for t in periodList:
                    if self.c_extDemand[i][t] > MyAbsTol:
                        dl.c_item_index[pos] = i
                        dl.c_period_index[pos] = t
                        dl.c_amounts[pos] = self.c_extDemand[i][t]
                        pos += 1
        else:
            print("Invalid choice!")
        return dl
    
    # generate demand list with specific sorting critieria (only for capactiy-based criteria)
    # returns demand list
    def generateDLdemand(self, str criterionPeriod, str criterionDemand, uint8 FirstPeriod=False):
        dl = DemandList(self.nonzerodemand)
        periodList = self.sortPeriods(criterionPeriod, FirstPeriod)
        cdef int pos = 0
        cdef int i
        for p in periodList:
            itemlist = []
            for it in range(0, self.items):
                itemlist.append(it)
            if criterionDemand == "D":
                itemlist.sort(key=lambda x: self.extDemand[x, p], reverse=True)
            elif criterionDemand == "DH":
                itemlist.sort(key=lambda x: self.extDemand[x, p] / self.holdingCost[x])
            elif criterionDemand == "DS":
                itemlist.sort(key=lambda x: self.extDemand[x, p] / self.setupCost[x])
            elif criterionDemand == "DHS":
                itemlist.sort(key=lambda x: self.extDemand[x, p] * self.holdingCost[x] / self.setupCost[x])
            elif criterionDemand == "DSH":
                itemlist.sort(key=lambda x: self.extDemand[x, p] / (self.holdingCost[x] * self.setupCost[x]))
            else:
                print("Invalid Criterion!")
            for i in itemlist:
                if self.c_extDemand[i][p] > MyAbsTol:
                    dl.c_item_index[pos] = i
                    dl.c_period_index[pos] = p
                    dl.c_amounts[pos] = self.c_extDemand[i][p]
                    pos += 1
        return dl
    
    # update the capacity to be blocked on all machines in all periods for production of not yet scheduled predecessors
    cdef update_blockedCap(self):
        self.c_blockedCap[:,:] = 0
        cdef int t
        cdef int i
        cdef int m
        for t in range(0,self.periods):
            for i in range(0,self.items):
                m = self.itemmachine[i]
                self.c_blockedCap[m][t] += self.c_projDemand[i][t]
                if self.c_projDemand[i][t] > 0:
                    self.c_blockedCap[m][t] += self.c_setupTime[m][i] / self.TBO[i]        # just a simple approximation, might be necessary to adapted
        for t in range(self.periods-1,0,-1):
            for m in range(0,self.machines):
                if self.c_blockedCap[m][t] > self.c_avCap[m][t]:
                    self.c_blockedCap[m][t-1] += self.c_blockedCap[m][t] - max(self.c_avCap[m][t], 0)
                    self.c_blockedCap[m][t] = max(self.c_avCap[m][t], 0)
    
    # reset projected demand
    cpdef reset_projDemand(self):
        self.c_projDemand[:][:] = 0.0
        self.c_blockedCap[:][:] = 0.0
    
    # update projected demand
    cdef update_projDemand(self, int t, int i, double amount):
        cdef int j
        for j in range(i+1,self.items):
            self.c_projDemand[j,t] += amount * self.c_BOM[j,i]
        self.update_blockedCap()
    
    # update projected demand (used only in shif of production routine) 
    cdef update_projDemandShift(self, int t_from, int t_to, int i, double amount):
        cdef int j
        for j in range(i,self.items):
            self.c_projDemand[j,t_to] += amount * self.c_BOM[j,i]
            self.c_projDemand[j,t_from] -= amount * self.c_BOM[j,i]
        self.update_blockedCap()
    
    # check the available capacity and take into account the blocked capacity
    # returns the amount that can be produced under capacity restriction
    cpdef checkCapacity(self, int i, int m, int t, double amount):
        cdef double maxamount = fmin(amount, max(0,self.c_avCap[m][t] - self.c_setupTime[m][i] * (1 - self.c_setup[i][t])-self.c_blockedCap[m][t]))
        requiredCap = np.zeros(self.machines)
        for pred in range(i+1, self.items):
            if self.BOM[pred][i] > 0:
                requiredCap[self.itemmachine[pred]] += maxamount * self.BOM[pred][i]
                requiredCap[self.itemmachine[pred]] += self.c_setupTime[self.itemmachine[pred]][pred] / self.TBO[pred]
        requiredCap[self.itemmachine[i]] += maxamount
        usableCap = np.zeros(self.machines)
        for mm in range(self.machines):
            usableCap[mm] += max(0, sum(self.avCap[mm][0:t+1]) - sum(self.blockedCap[mm][0:t+1]))
        usableCap[self.itemmachine[i]] -= self.c_setupTime[m][i] * (1 - self.c_setup[i][t])
        if usableCap[self.itemmachine[i]] < MyAbsTol:
            usableCap[self.itemmachine[i]] = 0
        percentage_maxamount = 1.0
        for mm in range(self.machines):
            if (requiredCap[mm] > usableCap[mm]):
                if usableCap[mm] == 0:
                    return 0
                else:
                    percentage_maxamount = fmin(percentage_maxamount, usableCap[mm]/requiredCap[mm])
        return maxamount * percentage_maxamount                
    
    # update the external demand for the current planning level
    cpdef update_extDemand(self, lotsizes):
        self.nonzerodemand = 0                                     # count non-zero demand elements
        self.extDemand = np.zeros((self.items, self.periods), dtype=float, order="C")
        self.c_extDemand = self.extDemand
        for j in range(self.items):
            if np.sum(lotsizes[j,:]) > MyAbsTol:
                for i in range(self.items):
                    if i not in self.thislevelitems:
                        if self.gozinto[i][j] > MyAbsTol:
                            self.c_extDemand[i, 0] += (lotsizes[j][0] - self.c_firstPeriodPlanned[j])*self.gozinto[i, j]
                            for t in range(1, self.periods):
                                self.c_extDemand[i, t] += lotsizes[j][t]*self.gozinto[i, j]
        for i in range(self.items):
            for t in range(0,self.periods):
                if (self.c_extDemand[i][t] > MyAbsTol):
                    self.nonzerodemand += 1
                    self.thislevelitems.append(i)
    
    # plan a demand list from pos1 till pos2
    # returns total cost of production plan
    cpdef double planDemandList(self, DemandList dl, uint8 resetSol = True, int pos1 = 0, int pos2 = 1000000000, uint8 resetPostDL = True):
        pos2 = min(pos2, self.nonzerodemand)
        if resetSol:                # reset production plan if true, otherwise continue with plan
            self.resetSolution()
        if resetPostDL:
            self.reset_postponementDL()
        cdef int pos
        for pos in range(pos1,pos2): # go through main demand list
            self.usePostDL = 1
            self.desactivatePostponement = False
            self.addDemand(dl.c_item_index[pos], dl.c_period_index[pos],  self.c_itemmachine[dl.c_item_index[pos]], dl.c_amounts[pos]) # add demand element to the partial production plan
            for pos_in_DL2 in range(0, self.nextPos): # go through postponement demand list
                self.keepInPostDL = False
                if self.usePostDL and (dl.c_item_index[pos] == self.postponementDL.c_item_index[pos_in_DL2]):
                    self.addDemand(self.postponementDL.c_item_index[pos_in_DL2], self.postponementDL.c_period_index[pos_in_DL2], self.c_itemmachine[self.postponementDL.c_item_index[pos_in_DL2]], self.postponementDL.c_amounts[pos_in_DL2], True, False)
                    if not self.keepInPostDL:
                        self.postponementDL.c_item_index[pos_in_DL2] = -1 # "delete" this element
                        
        # after we went through the whole main demand list, ensure that nothing remained unscheduled in the postponement demand list
        self.DL = 1 
        for pos in range(self.nextPos):
            if self.postponementDL.c_item_index[pos] == -1:
                pass
            else:
                self.desactivatePostponement = True
                self.addDemand(self.postponementDL.c_item_index[pos], self.postponementDL.c_period_index[pos], self.c_itemmachine[self.postponementDL.c_item_index[pos]], self.postponementDL.c_amounts[pos], True, False)
        return self.totalCost

# Run 2-SCH for Ap instances
In the following cell, we run 2-SCH with the reduced set of sorting rules and threshold value of 0.5, while shift of production and postponement routines are enabled.

The result is saved in a Pandas dataframe.

In [6]:
df = pd.DataFrame()
row = 0

for filename in file_names_Ap:
    print(filename)
    for pc in period_crit:
        for ic in cost_crit:
            for k in key_crit:
                rule = pc+'-'+ic+'-'+k
                if rule in reduced_set:
                    sol = MLCLSP(instance_path_Ap, filename, setuptimezero = True, storage = 1,
                                 shiftProduction=True, postponement = True, threshold=0.5)
                    more_levels = True
                    level = 0
                    while more_levels:
                        if level == 0:
                            dl = sol.generateDL(pc, ic, k)
                        else:
                            dl = sol.generateDL(pc, ic, k, FirstPeriod=True)                      
                        obj = sol.planDemandList(dl, resetSol=False)
                        sol.update_extDemand(sol.lotsizes)
                        sol.reset_projDemand()
                        if np.sum(sol.extDemand) < 1e-12:
                            more_levels = False
                        level += 1
                    df.loc[row, 'instance'] = filename
                    df.loc[row, 'Period'] = pc
                    df.loc[row, 'Item'] = ic
                    df.loc[row, 'Key'] = k
                    df.loc[row, 'RULE'] = rule
                    df.loc[row, 'OBJ'] = obj
                    df.loc[row, 'HC'] = sol.totalHoldingCost
                    df.loc[row, 'SC'] = sol.totalSetupCost
                    df.loc[row, 'OC'] = sol.totalOvertimeCost
                    row += 1        
    for pc in period_crit:
        for dc in capacity_crit:
            rule = pc+'-'+dc
            if rule in reduced_set:
                sol = MLCLSP(instance_path_Ap, filename, setuptimezero = True, storage = 1,
                             shiftProduction=True, postponement = True, threshold=0.5)
                more_levels = True
                level = 0
                while more_levels:
                    if level == 0:
                        dl = sol.generateDLdemand(pc, dc)
                    else:
                        dl = sol.generateDLdemand(pc, dc, FirstPeriod=True)
                    obj = sol.planDemandList(dl, resetSol=False)
                    sol.update_extDemand(sol.lotsizes)
                    sol.reset_projDemand()
                    if np.sum(sol.extDemand) < 1e-12:
                        more_levels = False
                    level += 1
                df.loc[row, 'instance'] = filename
                df.loc[row, 'Period'] = pc
                df.loc[row, 'Demand'] = dc
                df.loc[row, 'RULE'] = rule
                df.loc[row, 'OBJ'] = obj
                df.loc[row, 'HC'] = sol.totalHoldingCost
                df.loc[row, 'SC'] = sol.totalSetupCost
                df.loc[row, 'OC'] = sol.totalOvertimeCost
                row += 1        

Ap_G501130_MLCLS.dat
Ap_G501131_MLCLS.dat
Ap_G501132_MLCLS.dat
Ap_G501140_MLCLS.dat
Ap_G501141_MLCLS.dat
Ap_G501142_MLCLS.dat
Ap_G501230_MLCLS.dat
Ap_G501231_MLCLS.dat
Ap_G501232_MLCLS.dat
Ap_G501240_MLCLS.dat
Ap_G501241_MLCLS.dat
Ap_G501242_MLCLS.dat
Ap_G501330_MLCLS.dat
Ap_G501331_MLCLS.dat
Ap_G501332_MLCLS.dat
Ap_G501340_MLCLS.dat
Ap_G501341_MLCLS.dat
Ap_G501342_MLCLS.dat
Ap_G501430_MLCLS.dat
Ap_G501431_MLCLS.dat
Ap_G501432_MLCLS.dat
Ap_G501440_MLCLS.dat
Ap_G501441_MLCLS.dat
Ap_G501442_MLCLS.dat
Ap_G501530_MLCLS.dat
Ap_G501531_MLCLS.dat
Ap_G501532_MLCLS.dat
Ap_G501540_MLCLS.dat
Ap_G501541_MLCLS.dat
Ap_G501542_MLCLS.dat
Ap_G502130_MLCLS.dat
Ap_G502131_MLCLS.dat
Ap_G502132_MLCLS.dat
Ap_G502140_MLCLS.dat
Ap_G502141_MLCLS.dat
Ap_G502142_MLCLS.dat
Ap_G502230_MLCLS.dat
Ap_G502231_MLCLS.dat
Ap_G502232_MLCLS.dat
Ap_G502240_MLCLS.dat
Ap_G502241_MLCLS.dat
Ap_G502242_MLCLS.dat
Ap_G502330_MLCLS.dat
Ap_G502331_MLCLS.dat
Ap_G502332_MLCLS.dat
Ap_G502340_MLCLS.dat
Ap_G502341_MLCLS.dat
Ap_G502342_ML

In [7]:
df.head(30)

,instance,Period,Item,Key,RULE,OBJ,HC,SC,OC,Demand
0,Ap_G501130_MLCLS.dat,CUP,CUI,IP,CUP-CUI-IP,1.393034e+07,29451.189150,126640.0,1.377425e+07,NaN
1,Ap_G501130_MLCLS.dat,CUP,ES,IP,CUP-ES-IP,1.393034e+07,29451.189150,126640.0,1.377425e+07,NaN
2,Ap_G501130_MLCLS.dat,CUP,ESC,PI,CUP-ESC-PI,6.840651e+06,23288.558200,147600.0,6.669762e+06,NaN
3,Ap_G501130_MLCLS.dat,CUP,H,IP,CUP-H-IP,1.287400e+07,25235.683000,132080.0,1.271668e+07,NaN
4,Ap_G501130_MLCLS.dat,CUP,SH,PI,CUP-SH-PI,6.108734e+06,23915.539000,149920.0,5.934898e+06,NaN
5,Ap_G501130_MLCLS.dat,CUP,TBO,IP,CUP-TBO-IP,1.389543e+07,27399.449500,131360.0,1.373667e+07,NaN
6,Ap_G501130_MLCLS.dat,DECP,CUI,IP,DECP-CUI-IP,1.068249e+07,26485.004000,128240.0,1.052776e+07,NaN
7,Ap_G501130_MLCLS.dat,INCP,CUI,IP,INCP-CUI-IP,1.712848e+07,30977.998633,129040.0,1.696846e+07,NaN
8,Ap_G501130_MLCLS.dat,INCP,CUI,PI,INCP-CUI-PI,1.755265e+05,19366.519000,156160.0,0.000000e+00,NaN
9,Ap_G501130_MLCLS.dat,INCP,EC,IP,INCP-EC-IP,1.712848e+07,30977.998633,129040.0,1.696846e+07,NaN
